W tym zadaniu przećwiczymy PCA oraz poznamy inne metody redukcji wymiarowości.

W poprzednich dwóch zadaniach ekstrahowaliśmy cechy z sygnału podzielonego na ramki, uzyskując obraz (macierz 2D) opisujący sygnał. Jest to bardzo często stosowane w analizie sygnałów podejście, ale nie jedyne.

Z sygnałów można też ekstrahować inne parametry, np. parametry czasowe sygnału (częstotliwość przejść przez zero, środek ciężkości sygnału), parametry widmowe (momenty widmowe, kurtoza i skośność widma), parametry formantowe (częstotliwości formantów, stosunek ich amplitud) i wiele innych. Zbiór różnych parametrów sygnałów, które dają bardzo dobre rezultaty w klasyfikacji sygnałów akustycznych jest zawarty w bibliotece [openSMILE](https://audeering.github.io/opensmile/index.html).

OpenSMILE pozwala wyznaczyć predefiniowane zestawy parametrów opisujących dźwięk lub wideo - skorzystamy z zestawu 6373 parametrów opisujących pojedynczy sygnał (jest to zestaw cech i ich pochodnych używany w Interspeech Computational Paralinguistics Challenge; dla zainteresowanych opis: https://doi.org/10.3389/fpsyg.2013.00292 - tabele 2 i 3).

In [57]:
# jesli pracujemy w Colabie, konieczna jest instalacja biblioteki
# !pip install opensmile
import opensmile
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import librosa
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [58]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)

feats = smile.process_file('dane_testowe/1-phrase.wav')
feats.shape

(1, 6373)

Wykorzystując napisany wcześniej kod, wylicz parametry wszystkich sygnałów zawartych w pliku `phrase_files.csv`. Podziel dane na zbiór treningowy i testowy.

In [59]:
df = pd.read_csv('phrase_files.csv')
# dwa warianty odwołań do kolumny DataFrame:
files = df.file
y = df['label']

features_list = []
for f in files:
    path = os.path.join('dane_testowe',f)  # dostosuj ścieżkę jeśli pliki są gdzie indziej
    feats = smile.process_file(path)
    features_list.append(feats)
X = np.vstack(features_list)
print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

(20, 6373)


Ze względu na zróżnicowanie parametrów, musimy przeprowadzić standaryzację:

In [60]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Na ustandaryzowanych danych wytrenuj używany już wcześniej klasyfikator k najbliższych sąsiadów i wylicz metryki.

Warto ten krok zdefiniować jako funkcję - będziemy korzystać z niego kilka razy w tym notatniku.

In [61]:
def knn_classification(X_train, y_train, X_test, y_test):
    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    print(confusion_matrix(y_test, preds))
    print(classification_report(y_test, preds))

    return preds

preds = knn_classification(X_train, y_train, X_test, y_test)

[[3 0]
 [1 1]]
              precision    recall  f1-score   support

         0.0       0.75      1.00      0.86         3
         1.0       1.00      0.50      0.67         2

    accuracy                           0.80         5
   macro avg       0.88      0.75      0.76         5
weighted avg       0.85      0.80      0.78         5



6373 cechy opisujące jeden sygnał to dość dużo. Żeby zmniejszyć liczbę tych cech należy zastosować jedną z metod redukcji wymiarowości. Wpłynie to na zmniejszenie skomplikowania niektórych rodzajów klasyfikatorów, przyspieszy proces uczenia i pozwoli zmniejszyć przeuczenie modelu.

Zaczniemy od metody bazującej na wartości statystyki F. W bibliotece sklearn jest to funkcja [`SelectKBest`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html?highlight=selectkbest#sklearn.feature_selection.SelectKBest).

Wybierz liczbę cech, które chcesz zachować. Funkcja wybierze ze wszystkich 6373 cech tylko tyle, ile sprecyzujesz, w taki sposób, by jak najlepiej opisywać zmienność i różnicować obiekty.

Liczbę cech można optymalizować, ale tym zajmiemy się kiedy indziej. Dzisiaj spróbuj ręcznie dobrać ich liczbę tak, by wyniki klasyfikacji były zadowalające.

In [62]:
from sklearn.feature_selection import SelectKBest

liczba_cech = 100 #wpisz tu wartość
selector = SelectKBest(k=liczba_cech)
X_train_Kbest = selector.fit_transform(X_train, y_train)
X_test_Kbest = selector.transform(X_test)

c:\Users\mfox4\anaconda3\envs\pum\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [1125 1257 2156 2825 2962 3365 3520 3551 3582 3613 3644 3859 3874] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\mfox4\anaconda3\envs\pum\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Wytrenuj klasyfikator na zredukowanej liczbie cech. Jakie wartości metryk wyszły teraz? Czy są lepsze czy gorsze niż wcześniej?

In [63]:
preds = knn_classification(X_train_Kbest, y_train, X_test_Kbest, y_test)

[[3 0]
 [1 1]]
              precision    recall  f1-score   support

         0.0       0.75      1.00      0.86         3
         1.0       1.00      0.50      0.67         2

    accuracy                           0.80         5
   macro avg       0.88      0.75      0.76         5
weighted avg       0.85      0.80      0.78         5



Kolejną funkcją wartą poznania jest metoda bardzo podobna do `SelectKBest`, czyli [`SelectPercentile`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile). Działa na tej samej zasadzie, ale zamiast określać konkretną liczbę cech, które chcemy zachować, określamy jaka część cech ma być zachowana, np. 40%.

In [64]:
from sklearn.feature_selection import SelectPercentile

percentile = 20 #wpisz tu wartość
selector = SelectPercentile(percentile=percentile)
X_train_percent = selector.fit_transform(X_train, y_train)
X_test_percent = selector.transform(X_test)

preds = knn_classification(X_train_percent, y_train, X_test_percent, y_test)

[[2 1]
 [0 2]]
              precision    recall  f1-score   support

         0.0       1.00      0.67      0.80         3
         1.0       0.67      1.00      0.80         2

    accuracy                           0.80         5
   macro avg       0.83      0.83      0.80         5
weighted avg       0.87      0.80      0.80         5



c:\Users\mfox4\anaconda3\envs\pum\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [1125 1257 2156 2825 2962 3365 3520 3551 3582 3613 3644 3859 3874] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\mfox4\anaconda3\envs\pum\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Ostatnią metodą, którą dzisiaj poznamy jest [rekursywna eliminacja cech](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE) (RFE, ang. recursive feature elimination). Polega na tym, że estymator początkowo uczony jest na całym zbiorze danych i tworzony jest ranking cech w oparciu o to, jak bardzo są ważne podczas estymacji. Cechy najmniej istotne są usuwane i cały proces jest powtarzany. Usuwanie cech trwa tak długo, aż uzyskamy taką liczbę, jak sprecyzowana.

In [65]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

n_features = 100 #wpisz tu wartość
step = 10 #wpisz tu wartość
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=n_features, step=step) 
"""step - jeżeli >=1, to jest to liczba cech do usunięcia w danej iteracji,
jeżeli 0<step<1 - część cech do usunięcia.
Analogicznie działa parametr n_features_to_select"""
selector = selector.fit(X_train, y_train)
X_train_rfe = selector.transform(X_train)
X_test_rfe = selector.transform(X_test)


Powtórz proces uczenia i predykcji na cechach uzyskanych metodą RFE. Jakie wartości metryk wyszły tym razem? 

In [66]:
preds = knn_classification(X_train_rfe, y_train, X_test_rfe, y_test)

[[3 0]
 [2 0]]
              precision    recall  f1-score   support

         0.0       0.60      1.00      0.75         3
         1.0       0.00      0.00      0.00         2

    accuracy                           0.60         5
   macro avg       0.30      0.50      0.38         5
weighted avg       0.36      0.60      0.45         5



c:\Users\mfox4\anaconda3\envs\pum\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\mfox4\anaconda3\envs\pum\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\mfox4\anaconda3\envs\pum\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Na koniec wykonaj PCA na cechach z openSMILE, wykorzystując kod z poprzedniego zadania. Porównaj wyniki z pozostałymi metodami.

In [67]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.90)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

preds = knn_classification(X_train_pca, y_train, X_test_pca, y_test)

[[3 0]
 [2 0]]
              precision    recall  f1-score   support

         0.0       0.60      1.00      0.75         3
         1.0       0.00      0.00      0.00         2

    accuracy                           0.60         5
   macro avg       0.30      0.50      0.38         5
weighted avg       0.36      0.60      0.45         5



c:\Users\mfox4\anaconda3\envs\pum\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\mfox4\anaconda3\envs\pum\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\mfox4\anaconda3\envs\pum\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
